In [1]:
from keras.applications import VGG19
from keras.layers import Dense,Dropout,Flatten

Using TensorFlow backend.


In [2]:
from keras.applications import VGG19

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 64
img_cols = 64 

vgg16 = VGG19(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (64, 64, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

W0613 04:37:10.915172 4528539072 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0613 04:37:10.936594 4528539072 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0613 04:37:10.940976 4528539072 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0613 04:37:10.992564 4528539072 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0613 04:37:11.6423

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 Conv2D False
11 MaxPooling2D False
12 Conv2D False
13 Conv2D False
14 Conv2D False
15 Conv2D False
16 MaxPooling2D False
17 Conv2D False
18 Conv2D False
19 Conv2D False
20 Conv2D False
21 MaxPooling2D False


In [5]:
#Adding Model on TOp
from keras.models import Model
def addTopModel(bottom_model,num_classes,D=256):
    top_model = bottom_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model= Dense(D,activation="relu")(top_model)
    top_model= Dropout(0.3)(top_model)
    top_model= Dense(units=17,activation="softmax")(top_model)
    return top_model

In [6]:
#Adding top and bottom model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17

FC_Head = addTopModel(vgg16, num_classes=num_classes)

model = Model(inputs = vgg16.input , outputs = FC_Head)


print(model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './train'
validation_data_dir = './validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 8
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [11]:
#Training Top LAyer
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("flowers_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 5
batch_size = 64

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
from keras.models import load_model

model.save("flower_vgg.h5")

Epoch 1/5
18/18 [==============================] - 16s 912ms/step - loss: 1.6485 - acc: 0.4375 - val_loss: 2.0019 - val_acc: 0.4500

Epoch 00001: val_loss improved from inf to 2.00192, saving model to flowers_vgg.h5
Epoch 2/5
18/18 [==============================] - 17s 933ms/step - loss: 1.6630 - acc: 0.4792 - val_loss: 1.3906 - val_acc: 0.5500

Epoch 00002: val_loss improved from 2.00192 to 1.39061, saving model to flowers_vgg.h5
Epoch 3/5
18/18 [==============================] - 15s 851ms/step - loss: 1.3129 - acc: 0.5304 - val_loss: 2.5852 - val_acc: 0.4000

Epoch 00003: val_loss did not improve from 1.39061
Epoch 4/5
18/18 [==============================] - 18s 1s/step - loss: 1.3229 - acc: 0.5764 - val_loss: 1.2145 - val_acc: 0.7500

Epoch 00004: val_loss improved from 1.39061 to 1.21452, saving model to flowers_vgg.h5
Epoch 5/5
18/18 [==============================] - 19s 1s/step - loss: 1.1828 - acc: 0.5972 - val_loss: 0.3017 - val_acc: 0.9000

Epoch 00005: val_loss improved fr